In [ ]:
import torch
from torch_geometric.data import Data
import torch_geometric as tg
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import HeatMap
from shapely.geometry import Point, LineString, Polygon
import shapely
import momepy as mp 
import esda
import seaborn as sns
from shapely.strtree import STRtree
import pickle
from tqdm import tqdm

import os, glob, sys

print(os.path.exists('../scripts/bike_functions.py'))
import sys
sys.path.append('../scripts')
from bike_functions import *

sns.set_theme()

feat_set_names = {
    17: 'aerialway aeroway barrier boundary building craft emergency healthcare historic landuse leisure military natural office place power public_transport railway route service shop tourism water waterway'.split(),
    21: 'aeroway barrier boundary building craft emergency historic landuse leisure military natural office place public_transport railway service tourism water'.split(),
    23: 'aeroway barrier boundary building office place public_transport railway route service shop tourism'.split(),
    24: 'building natural shop'.split(),
}

with open(f'../data/graphs/28/linegraph_tg.pkl', 'rb') as f:
    data = pickle.load(f)

from copy import deepcopy

for i in feat_set_names:
    lg = deepcopy(data)

    # Create a new feature set
    base_feats = ['bc', 'width', 'length', 'maxspeed', 'lanes', 'oneway', 'reversed']
    requested_feats = base_feats + feat_set_names[i]

    # Safely find valid indices and their names
    valid_indices = []
    valid_feat_names = []
    for feat in requested_feats:
        if feat in lg.feat_names:
            idx = lg.feat_names.index(feat)
            valid_indices.append(idx)
            valid_feat_names.append(feat)

    # Subset the feature matrix and sync feat_names
    lg.x = lg.x[:, valid_indices]
    lg.feat_names = valid_feat_names

    print(f"Feature set {i}:")
    print("  Features:", lg.feat_names)
    print("  Shape:", lg.x.shape)

    # Save the new feature set
    os.makedirs(f'../data/graphs/{i}', exist_ok=True)
    with open(f'../data/graphs/{i}/linegraph_tg.pkl', 'wb') as f:
        pickle.dump(lg, f)
